In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.callbacks import ReduceLROnPlateau


2024-07-17 01:00:35.280199: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-17 01:00:35.435529: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-17 01:00:36.621536: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [19]:
# Carrega a base de dados a partir de seu caminho
data = np.load('dataset.npz', allow_pickle=True)
X_test = data['X_test']
X_train = data['X_train']
y_test = data['y_test']
y_train = data['y_train']


In [24]:
data['y_test']

array(None, dtype=object)

In [ ]:
# Scale
scaler = StandardScaler()
X_train_sc = scaler.fit_transform(X_train)
X_test_sc = scaler.transform(X_test)

In [58]:
#Model
initializer = keras.initializers.GlorotNormal(seed=12227)

inputs = keras.layers.Input(shape=(X_test_sc.shape[1],))

n = 128
H = keras.layers.Dense(units=n, activation='relu', kernel_initializer=initializer)(inputs)
shortcut = H
H = keras.layers.Dense(units=n*2, activation='relu', kernel_initializer=initializer)(H)
H = keras.layers.Dense(units=n*4, activation='relu', kernel_initializer=initializer)(H)
H = keras.layers.Dense(units=n*8, activation='relu', kernel_initializer=initializer)(H)
H = keras.layers.Dense(units=n*4, activation='relu', kernel_initializer=initializer)(H)
H = keras.layers.Dense(units=n*2, activation='relu', kernel_initializer=initializer)(H)
H = keras.layers.Dense(units=n, activation='relu', kernel_initializer=initializer)(H)
H=H+shortcut
outputs = keras.layers.Dense(units=1, activation='sigmoid', kernel_initializer=initializer)(H)

model = keras.models.Model(inputs=inputs, outputs=outputs)

In [59]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])

In [60]:
def lr_schedule(epoch, lr):
    if epoch < 10:
        return 0.01
    elif epoch < 20:
        return 0.001
    elif epoch < 30:
        return 0.0001
    else:
        return 0.0001/epoch
    
scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
    
history = model.fit(X_train_sc, y_train, epochs=50, batch_size=256, callbacks=[scheduler])

Epoch 1/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - binary_accuracy: 0.7621 - loss: 0.6435 - learning_rate: 0.0100
Epoch 2/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - binary_accuracy: 0.8099 - loss: 0.4013 - learning_rate: 0.0100
Epoch 3/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - binary_accuracy: 0.8113 - loss: 0.4022 - learning_rate: 0.0100
Epoch 4/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - binary_accuracy: 0.8120 - loss: 0.4009 - learning_rate: 0.0100
Epoch 5/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - binary_accuracy: 0.8138 - loss: 0.3998 - learning_rate: 0.0100
Epoch 6/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - binary_accuracy: 0.8149 - loss: 0.3951 - learning_rate: 0.0100
Epoch 7/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - binary_accuracy: 0.8184 - loss: 0.3928 - learning_rate: 0.0100
Epoch 8/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - binary_accuracy: 0.8164 - loss: 0.3925 - learning_rate: 0.0100
Epoch 9/50
187/187 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - b

In [55]:
y_pred_prob = model.predict(X_test_sc)
y_pred = (y_pred_prob > 0.5).astype(int)

3152/3152 ━━━━━━━━━━━━━━━━━━━━ 3s 850us/step


In [56]:
one_d_list = [item[0] for item in y_pred.tolist()]

In [57]:

# Gera um DataFrame no formato esperado da submissão
num_samples = X_test.shape[0]
submission_df = pd.DataFrame({
    'ID': np.arange(1, num_samples + 1),
    'Prediction': one_d_list
})

# Salva o arquivo CSV no diretório atual
submission_df.to_csv('submission2.csv', index=False)